# 🏭 Fabrication de Dataset pour Timer SF6

Ce notebook contient les étapes pour créer un dataset d'entraînement à partir de vidéos ou frames SF6.

## Flux de travail :
1. **Extraction de frames** depuis une vidéo SF6
2. **Découpe manuelle** des chiffres du timer 
3. **Étiquetage assisté** pour créer le dataset final dans `../input/learning/dataset/`

⚠️ **Note** : Ces étapes ne sont nécessaires que si vous n'avez pas encore de dataset prêt.

## 1 – Extraction des frames depuis une vidéo

In [ ]:
import cv2
import os

video_path = "../input/match.mp4"  # Nouveau chemin
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps // 10)  # ~10 images/sec

count, saved = 0, 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if count % frame_interval == 0:
        frame_name = os.path.join(output_dir, f"frame_{saved:04d}.png")
        cv2.imwrite(frame_name, frame)
        saved += 1
    count += 1

cap.release()
print(f"✅ {saved} frames sauvegardées dans {output_dir}")

## 2 – Découpe du timer

In [ ]:
import matplotlib.pyplot as plt

digit_boxes = [
    (860, 40, 40, 80),  # Chiffre 1
    (905, 40, 40, 80),  # Chiffre 2
]

frames_list = sorted(os.listdir(output_dir))
sample_frame = cv2.imread(os.path.join(output_dir, frames_list[0]))

plt.imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
plt.title("Frame complète - Timer visible")
plt.show()

for (x, y, w, h) in digit_boxes:
    crop = sample_frame[y:y+h, x:x+w]
    plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    plt.show()

## 3 – Étiquetage assisté du dataset

In [ ]:
from PIL import Image
import shutil
import os

dataset_dir = "../input/learning/dataset"
os.makedirs(dataset_dir, exist_ok=True)

for d in range(10):
    os.makedirs(os.path.join(dataset_dir, str(d)), exist_ok=True)

frames_list = sorted(os.listdir(output_dir))

print(f"📁 Sauvegarde du dataset dans: {dataset_dir}")
print(f"🏗️  Structure: 0/, 1/, 2/, ... 9/ créées")

for frame_name in frames_list:
    frame_path = os.path.join(output_dir, frame_name)
    frame = cv2.imread(frame_path)
    for i, (x, y, w, h) in enumerate(digit_boxes):
        crop = frame[y:y+h, x:x+w]
        plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        label = input(f"Chiffre #{i+1} : ")
        if label.isdigit() and 0 <= int(label) <= 9:
            save_path = os.path.join(dataset_dir, label, f"{frame_name}_digit{i}.png")
            cv2.imwrite(save_path, crop)
            print(f"✅ Sauvé: {save_path}")
        else:
            print(f"⚠️  Ignoré (label invalide): {label}")

print(f"\n🎯 Étiquetage terminé! Dataset disponible dans {dataset_dir}")
print(f"💡 Vous pouvez maintenant utiliser le notebook Timer/timer_training.ipynb")